In [2]:
import requests 
import json
import os
from dotenv import load_dotenv
import pandas as pd
from pandas import json_normalize
import tweepy
import time
load_dotenv()

True

In [3]:
stone = pd.read_excel("../data/musicbrainz.xlsx")

In [14]:
stone.head(10)

,Number,Year,Album,Artist,Type,Rating,Releases,Genre,Gen,Subgenre
0,1,1971,What's Going On,Marvin Gaye,Album,4.65,38,r&b,r&b,NaN
1,2,1966,Pet Sounds,The Beach Boys,Album,4.45,47,rock,rock,NaN
2,3,1971,Blue,Joni Mitchell,Album,4.45,18,folk,folk,NaN
3,4,1976,Songs in the Key of Life,Stevie Wonder,Album,4.55,24,soul,soul,NaN
4,5,1969,Abbey Road,The Beatles,Album,4.70,54,rock,rock,NaN
5,6,1991,Nevermind,Nirvana,Album,4.45,68,grunge,grunge,NaN
6,7,1977,Rumours,Fleetwood Mac,Album,4.60,59,pop,pop,NaN
7,8,1984,Purple Rain,Prince and The Revolution,Album + Soundtrack,4.25,36,rock,rock,NaN
8,9,1975,Blood on the Tracks,Bob Dylan,Album,4.35,25,folk,folk,NaN
9,10,1998,The Miseducation of Lauryn Hill,Lauryn Hill,Album,4.15,13,r&b,r&b,NaN


In [6]:
stone.isna().sum()

Number       0
Year         0
Title        0
Artist       0
Type         0
Rating      38
Releases     0
Genre        0
dtype: int64

In [7]:
stone.rename(columns={"Title": "Album"}, inplace = True)

In [8]:
stone.Album = stone.Album.replace(1999,"1999",regex=True)
stone.Album = stone.Album.replace(21,"21",regex=True)
stone.Album = stone.Album.replace(1989,"1989",regex=True)

In [9]:
stone.Artist = stone.Artist.str.strip()
stone.Album = stone.Album.str.strip()
stone.Type = stone.Type.str.strip()
stone.Genre = stone.Genre.str.strip()

In [10]:
stone["Gen"] = stone["Genre"].str.extract(r"(^([^,])+)")[0]
stone["Subgenre"] = stone["Genre"].str.extract(r",\s*(.*$)")[0]

In [13]:
stone.Album = stone.Album.str.replace(r'(Metallica...The Black Album..)',"Metallica",regex=True)
stone.Album = stone.Album.str.replace(r'(Eagles...st album.)',"Eagles",regex=True)
stone.Artist = stone.Artist.str.replace("Neil Young with Crazy Horse","Neil Young & Crazy Horse",regex=True)
stone.Artist = stone.Artist.str.replace(r"(Bob Dylan...The Band)","Bob Dylan And The Band",regex=True)
stone.Artist = stone.Artist.str.replace("Prince and The Revolution","Prince & The Revolution",regex=True)
stone.Artist = stone.Artist.str.replace(r"(Rufus...Chaka Khan)","Rufus",regex=True)
stone.Album = stone.Album.str.replace(r'(Proud Mary.*)',"Proud Mary",regex=True)

In [39]:
stone["Alb_url"] = stone["Album"]
stone["Art_url"] = stone["Artist"]

In [40]:
stone.Alb_url = stone.Alb_url.str.replace(r'(^The Beatles\s..The White Album..)',"The Beatles (Remastered)",regex=True)
stone.Alb_url = stone.Alb_url.str.replace(r'(^The Band...The Brown Album..)',"The Band",regex=True)

In [41]:
stone["Alb_url"] = stone["Alb_url"].str.replace("'","%27",regex=True)
stone["Alb_url"] = stone["Alb_url"].str.replace("&","%26",regex=True)
stone["Alb_url"] = stone["Alb_url"].str.replace(".","%2e",regex=True)
stone["Alb_url"] = stone["Alb_url"].str.replace("/","%2f",regex=True)
stone["Alb_url"] = stone["Alb_url"].str.replace("#","%23",regex=True)
stone["Alb_url"] = stone["Alb_url"].str.replace(r"(\s+)","%20",regex=True)

stone["Art_url"] = stone["Art_url"].str.replace("'","%27",regex=True)
stone["Art_url"] = stone["Art_url"].str.replace("&","%26",regex=True)
stone["Art_url"] = stone["Art_url"].str.replace(".","%2e",regex=True)
stone["Art_url"] = stone["Art_url"].str.replace("/","%2f",regex=True)
stone["Art_url"] = stone["Art_url"].str.replace("#","%23",regex=True)
stone["Art_url"] = stone["Art_url"].str.replace(r"(\s+)","%20",regex=True)

In [42]:
stone.

,Number,Year,Album,Artist,Type,Rating,Releases,Genre,Gen,Subgenre,Alb_url,Art_url
0,1,1971,What's Going On,Marvin Gaye,Album,4.65,38,r&b,r&b,NaN,What%27s%20Going%20On,Marvin%20Gaye
1,2,1966,Pet Sounds,The Beach Boys,Album,4.45,47,rock,rock,NaN,Pet%20Sounds,The%20Beach%20Boys
2,3,1971,Blue,Joni Mitchell,Album,4.45,18,folk,folk,NaN,Blue,Joni%20Mitchell
3,4,1976,Songs in the Key of Life,Stevie Wonder,Album,4.55,24,soul,soul,NaN,Songs%20in%20the%20Key%20of%20Life,Stevie%20Wonder
4,5,1969,Abbey Road,The Beatles,Album,4.70,54,rock,rock,NaN,Abbey%20Road,The%20Beatles
...,...,...,...,...,...,...,...,...,...,...,...,...
495,496,1998,¿Dónde están los ladrones?,Shakira,Album,4.50,4,pop,pop,NaN,¿Dónde%20están%20los%20ladrones?,Shakira
496,497,1985,The Indestructible Beat of Soweto,Various Artists,Album + Compilation,NaN,3,"mbaqanga , south african",mbaqanga,south african,The%20Indestructible%20Beat%20of%20Soweto,Various%20Artists
497,498,1977,Suicide,Suicide,Album,4.00,10,rock,rock,NaN,Suicide,Suicide
498,499,1977,Ask Rufus,Rufus,Album,NaN,2,r&b,r&b,NaN,Ask%20Rufus,Rufus


In [44]:
stone.to_csv("../data/stone.csv")